In [15]:
# https://www.tensorflow.org/get_started/get_started

import tensorflow as tf    

In [16]:
# tensor表示张量，一二三维张量 分别为 标量 向量 矩阵。更高维张量很少使用。
# 张量可以是常量、变量（作为参数）、占位符（作为输入）
n1 = tf.constant(3.14) # 一维张量
# session用来执行一个函数或者优化器，作为Python和C++ Backend的中介。
s = tf.Session()
print s.run([n1])
n2 = tf.constant(9.8)
# 创建一个函数
op1 = tf.add(n1, n2)
print s.run(op1)

p1 = tf.placeholder(tf.float32)
p2 = tf.placeholder(tf.float32)
p_add = p1 + p2
print s.run(p_add, {p1: 3.14, p2: 9.8})

W = tf.Variable([3.], dtype=tf.float32)
b = tf.Variable([2.], dtype=tf.float32)
x = tf.placeholder(tf.float32)
lr = W * x + b
init = tf.global_variables_initializer()
s.run(init)
print s.run(lr, {x: [1,2,3,4,5]})

y = tf.placeholder(tf.float32)
diff = tf.square(y - lr)
loss = tf.reduce_sum(diff)
data = [1,2,3]
labels = [4,3,2]
print s.run(loss, {x: data, y: labels})

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
for i in range(1000):
    s.run(train, {x: data, y: labels})
print s.run([W, b])
print s.run(loss, {x: data, y: labels})




[3.1400001]
12.94
12.94
[  5.   8.  11.  14.  17.]
107.0
[array([-0.99873865], dtype=float32), array([ 4.99713278], dtype=float32)]
3.5387e-06


In [6]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=299)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/var/folders/t4/8039y7kx1qxfwzdkd74ynx2m0000gn/T/tmpkAknxd', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/t4/8039y7kx1qxfwzdkd74ynx2m0000gn/T/tmpkAknxd/model.ckpt.
INFO:tensorflow:loss = 9.0, step = 1
INFO:tensorflow:global_step/sec: 644.023
INFO:tensorflow:loss = 0.220711, step = 101 (0.157 sec)
INFO:tensorflow:global_step/sec: 877.084
INFO:tensorflow:loss = 0.00646984, step = 201 (0.114 sec)
INFO:tensorflow:Saving checkpoints for 299 into /var/folders/t4/8039y7kx1qxfwzdkd74ynx2m0000gn/T/tmpkAknxd/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00111338.
INFO:tensorflow:Starting evaluation at 2017-12-29-08:16:

In [14]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/var/folders/t4/8039y7kx1qxfwzdkd74ynx2m0000gn/T/tmpHGxUwx', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/t4/8039y7kx1qxfwzdkd74ynx2m0000gn/T/tmpHGxUwx/model.ckpt.
INFO:tensorflow:loss = 8.66328258808, step = 1
INFO:tensorflow:global_step/sec: 932.837
INFO:tensorflow:loss = 0.00656824125986, step = 101 (0.109 sec)
INFO:tensorflow:global_step/sec: 898.174
INFO:tensorflow:loss = 0.000355017149994, step = 201 (0.111 sec)
INFO:tensorflow:global_step/sec: 927.609
INFO:tensorflow:loss = 0.000379241032489, step = 301 (0.108 sec)
INFO:tensorflow:global_step/sec: 914.411
INFO:tensorflow:loss = 3.71038700157e-05, step = 401

In [ ]:

# Tutorial2
# https://www.tensorflow.org/get_started/mnist/beginners



In [23]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


def main():
  # Import data
  from tensorflow.examples.tutorials.mnist import input_data
  mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

  # Create the model
  x = tf.placeholder(tf.float32, [None, 784]) # Here None means that a dimension can be of any length.
  W = tf.Variable(tf.zeros([784, 10]))
  b = tf.Variable(tf.zeros([10]))
  y = tf.matmul(x, W) + b

  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])

  # The raw formulation of cross-entropy,
  #
  #   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
  #                                 reduction_indices=[1]))
  #
  # can be numerically unstable.
  #
  # So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
  # outputs of 'y', and then average across the batch.
  cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
  train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

  sess = tf.InteractiveSession()
  tf.global_variables_initializer().run()
  # Train
  for _ in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

  # Test trained model
  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print("accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

if __name__ == '__main__':
  main()

    
    
    
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
accuracy: 0.921


In [19]:
import tensorflow as tf
r = tf.argmax([[1,2,3],[4,3,2]], 1)
s = tf.Session()
s.run(r)



array([2, 0])

In [24]:


# Tutorial3 https://www.tensorflow.org/get_started/mnist/pros


In [40]:
# guide: https://www.tensorflow.org/programmers_guide/tensors
import tensorflow as tf


s = tf.Session()
exe = lambda v: print(s.run(v))
t = tf.zeros([1,2,3,4,])  # tensor是immutable的
    # PlaceHolder Variables Constant SparseTensor 都是 Tensor
    # Tensor是一个计算单元，有input和output
    # 多个tensor连接组成graph
    # eval/run的过程就是计算graph的过程
exe(tf.rank(t))  # 多少维的，例如矩阵是2维的
exe(tf.shape(t))  # 每个维度的size

t = tf.reshape(t, [6, 4])
t.eval()  # 使用default session来run这个tensor. 返回一个 numpy array.

# tensor的执行可能依赖context，例如 PlaceHolder
p = tf.placeholder(tf.float32)
p.eval(feed_dict={p:3.14})


4
[1 2 3 4]


array(3.140000104904175, dtype=float32)

In [68]:
# guide: https://www.tensorflow.org/programmers_guide/variables

a = tf.Variable(3.14, tf.float32)
b = tf.placeholder(tf.float32)
a = tf.Print(a, [a, b])  # 当eval a的时候，打印 a 和 b 的值。
c = a + b
s = tf.Session()

s.run(tf.global_variables_initializer())  # 初始化session中涉及到的variables
    # variable初始化的时候指定了初始值，估计是要在gpu或者其他地方用指定的初始值进行初始化操作
print(s.run(c, feed_dict={b: 6.28}))

tf.global_variables_initializer().run()  # 初始化default session
print(c.eval(feed_dict={b: 6.28}))  # 貌似没有Print出来



9.42
9.42


In [86]:
var = tf.get_variable("my_var7", [2, 3], dtype=tf.float32, initializer=tf.zeros_initializer)  
    # 创建指定名称的变量，维度为 2x3. 
    # 这个变量会一直存在。后续再创建同名变量会报错。
print(s.run(tf.report_uninitialized_variables()))
tf.global_variables_initializer().run()
var.eval()

print(tf.GraphKeys.GLOBAL_VARIABLES)  # 设备间共享的变量
print(tf.GraphKeys.TRAINABLE_VARIABLES)  # 可参与训练的变量
print(tf.GraphKeys.LOCAL_VARIABLES)  # 不可参与训练的变量

print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))


ValueError: Variable my_var7 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-84-9e1279d39543>", line 1, in <module>
    var = tf.get_variable("my_var7", [2, 3], dtype=tf.float32, initializer=tf.zeros_initializer)
  File "//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [ ]:


# 自定义estimator: https://www.tensorflow.org/get_started/estimator




    